# 07 - MMRM Mixed model repeated measures 

## Data

Source of data: SPR, an R package to simulate clinical data as part of training in R statistical programming.

https://cjangelo.github.io/SPR/index.html

Data generated by the program in https://cjangelo.github.io/SPR/articles/a08_Fit_MMRM_in_R.html

Dataset df07.csv

In [13]:
library(readr)
df07 <- read_csv("data/df07.csv",
                 show_col_types = FALSE)
head(df07)


USUBJID,Group,Time,Y_comp,XB,error,Y_mar
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
Subject_0001,Group_1,Time_1,-0.07673563,0,-0.07673563,-0.0767356342764288
Subject_0002,Group_2,Time_1,-0.41197323,0,-0.41197323,-0.411973233779806
Subject_0003,Group_1,Time_1,-1.22969052,0,-1.22969052,-1.22969052255179
Subject_0004,Group_2,Time_1,0.03483386,0,0.03483386,0.0348338577296185
Subject_0005,Group_1,Time_1,0.51250275,0,0.51250275,0.512502751066066
Subject_0006,Group_2,Time_1,0.23896566,0,0.23896566,0.238965657323612


## SAS program snippet

The following SAS code will be executed.

## Results

The output is divided into blocks to explain it and to reproduce it afterwards in the different languages.

### Block 1
![Block 1](img_screenshots/block_101.png)

This block gives information about the model.

### R chunk for reproduction

The first chunk uses nlme::gls.


In [14]:
library(SPR)
library(MASS)
library(glmmTMB) # https://cran.r-project.org/web/packages/glmmTMB/vignettes/covstruct.html
library(emmeans)
library(nlme)
library(lme4)
library(broom)
my_gls <- nlme::gls(Y_comp ~ Group + Time + Group*Time,
                       data = df07,
                       correlation = corSymm(form = ~ 1 | USUBJID),    #  unstructured correlation
                       weights = varIdent(form = ~ 1 | Time),          #  freely estimate variance at subsequent timepoints
                       na.action = na.exclude)
summary(my_gls)

Generalized least squares fit by REML
  Model: Y_comp ~ Group + Time + Group * Time 
  Data: df07 
     AIC      BIC    logLik
  1036.4 1107.883 -500.2002

Correlation Structure: General
 Formula: ~1 | USUBJID 
 Parameter estimate(s):
 Correlation: 
  1     2     3    
2 0.779            
3 0.574 0.728      
4 0.484 0.535 0.763
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | Time 
 Parameter estimates:
  Time_1   Time_2   Time_3   Time_4 
1.000000 1.095606 1.217203 1.499564 

Coefficients:
                             Value  Std.Error    t-value p-value
(Intercept)             -0.0693893 0.13692310 -0.5067758  0.6126
GroupGroup_2             0.3336959 0.19363850  1.7232931  0.0856
TimeTime_2              -0.0531747 0.09614668 -0.5530579  0.5805
TimeTime_3               0.0054841 0.14258851  0.0384609  0.9693
TimeTime_4              -0.0566736 0.18351241 -0.3088272  0.7576
GroupGroup_2:TimeTime_2  0.0280852 0.13597194  0.2065512  0.8365
GroupGroup

The second chunk uses mmrm().

mmrm need categorical variables as factors.

In [15]:
library(mmrm)
df08 <- df07
df08$Group_fct <- as.factor(df08$Group)
df08$Time_fct <- as.factor(df08$Time)
my_mmrm <- mmrm(
  formula = Y_comp ~ Group_fct + Time_fct + Group_fct*Time_fct + us(Time_fct | USUBJID),
  data = df08
)
summary(my_mmrm)

mmrm fit

Formula:     
Y_comp ~ Group_fct + Time_fct + Group_fct * Time_fct + us(Time_fct |  
    USUBJID)
Data:        df08 (used 400 observations from 100 subjects with maximum 4 
timepoints)
Covariance:  unstructured (10 variance parameters)
Method:      Satterthwaite
Inference:   REML

Model selection criteria:
     AIC      BIC   logLik deviance 
  1020.4   1046.5   -500.2   1000.4 

Coefficients: 
                                 Estimate Std. Error        df t value Pr(>|t|)
(Intercept)                     -0.069389   0.136924 98.010000  -0.507    0.613
Group_fctGroup_2                 0.333696   0.193640 98.010000   1.723    0.088
Time_fctTime_2                  -0.053175   0.096145 98.010000  -0.553    0.581
Time_fctTime_3                   0.005484   0.142592 98.000000   0.038    0.969
Time_fctTime_4                  -0.056674   0.183520 97.990000  -0.309    0.758
Group_fctGroup_2:Time_fctTime_2  0.028085   0.135969 98.010000   0.207    0.837
Group_fctGroup_2:Time_fctTime_3 

### Block 2
![Block 2](img_screenshots/block_102.png)

This block gives the level for the categorical variables.

The right section of the screenshot was removed to fit this page.

### R chunk for reproction

In [16]:
table(df07$Group)
table(df07$Time)
head(table(df07$USUBJID))


Group_1 Group_2 
    200     200 


Time_1 Time_2 Time_3 Time_4 
   100    100    100    100 


Subject_0001 Subject_0002 Subject_0003 Subject_0004 Subject_0005 Subject_0006 
           4            4            4            4            4            4 

### Block 3
![Block 3](img_screenshots/block_103.png)

This block gives information about the number of subjects and the maximal number of observations per subject.

### R chunk for reproduction


In [17]:
suppressPackageStartupMessages(library(tidyverse))
print("Subjects:")
df07 %>% select(USUBJID) %>% unique() %>% count(name = "Subjects") %>% pull(Subjects)
print("Max Obs per Subject")
df07 %>% select(USUBJID, Time) %>% group_by(USUBJID) %>%
summarize(MaxSubObs = n()) %>% 
ungroup() %>%
summarize(MaxObs = max(MaxSubObs)) %>%
pull(MaxObs)

[1] "Subjects:"


[1] 100

[1] "Max Obs per Subject"


[1] 4

### Block 4
![Block 4](img_screenshots/block_104.png)

The number of observations used might be less than the number of observations read. SAS performs a listwise deletion (complete case analysis) if missing values are present.

# Todo: Complete the following cells

### R chunk for reproduction


### Block 5
![Block 5](img_screenshots/block_105.png)

This block gives information about the iteration process.

### R chunk for reproduction


### Block 6
![Block 6](img_screenshots/block_106.png)

This block informs about the status of the iterative estimation process at the end of the Newton-Raphson optimization.

### R chunk for reproduction

### Block 7
![Block 7](img_screenshots/block_107.png)

Details for this block can be found in the SAS Proc MIXED manuals in Mixed Models Theory.

### R chunk for reproduction


### Block 8
![Block 8](img_screenshots/block_108.png)

This block gives statistics about the estimated mixed models.

### R chunk for reproduction



### Block 9
![Block 9](img_screenshots/block_109.png)

This block gives the likelihood ratio test for the null model.

### R chunk for reproduction


### Block 10
![Block 10](img_screenshots/block_110.png)

This block provides the estimates for the fixed effects of the model.

### R chunk for reproduction



In [18]:
data.frame("estimates" = my_gls$coeff)

,estimates
,<dbl>
(Intercept),-0.069389313
GroupGroup_2,0.333695897
TimeTime_2,-0.053174683
TimeTime_3,0.005484079
TimeTime_4,-0.056673629
GroupGroup_2:TimeTime_2,0.028085170
GroupGroup_2:TimeTime_3,-0.126215247
GroupGroup_2:TimeTime_4,-0.146804525


### Block 11
![Block 11](img_screenshots/block_111.png)

This block contains hypothesis tests for the significance of each of the fixed effects.

### R chunk for reproduction